In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import importlib
import lightgbm as lgb
import optuna
#import airbnb_test_func_ver2 as tf
#importlib.reload(tf)

%matplotlib inline

In [6]:
df = pd.read_csv("train_meanprice.csv")

#df = tf.data_pre(df)

#model = tf.model_lgb(df)

In [11]:
def objective(trial):

    df_train, df_val = train_test_split(df, test_size=0.2)
    
    col = "n"
    train_y = df_train[col]
    train_x = df_train.drop(col, axis=1)

    val_y = df_val[col]
    val_x = df_val.drop(col, axis=1)

    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y)

    #https://lightgbm.readthedocs.io/en/latest/Parameters.html
    #https://knknkn.hatenablog.com/entry/2021/06/29/125226
    #https://zenn.dev/megane_otoko/articles/2021ad_09_optuna_optimization
    
    
    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 50)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    
    params = {
        "task": "train", 
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": {"rmse"}, 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 123, 
        "max_depth": max_depth,
        "num_iterations": 100      
    }

        #カテゴリカルデータをリストで渡す
    categorical_list = [
                        "Month",
                        "shopID",
                        "Cat",
                        "itemID", 
                        ]
    #https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html
    #https://lightgbm.readthedocs.io/en/latest/Python-API.html
    model = lgb.train(params, 
                        trains, 
                        valid_sets=valids, 
                        categorical_feature=categorical_list, 
                        num_boost_round=1000, 
                        )

    pred_y = model.predict(val_x)
    rmse = np.sqrt(mse(val_y, pred_y))
    
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2023-09-06 17:49:22,141] A new study created in memory with name: no-name-0fa87fbb-5e12-408b-8161-a5ca9f80b6dd
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.751615
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[I 2023-09-06 17:49:24,322] Trial 0 finished with value: 1.0813957565304533 and parameters: {'learning_rate': 0.21729665284754485, 'num_leaves': 43, 'tree_learner': 'feature', 'lambda_l1': 174.48183514417087, 'lambda_l2': 49.246453447757645, 'max_depth': 11}. Best is trial 0 with value: 1.0813957565304533.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 456
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.746435
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[I 2023-09-06 17:49:24,904] Trial 1 finished with value: 1.3658079701715253 and parameters: {'learning_rate': 0.07146808728458331, 'num_leaves': 14, 'tree_learner': 'serial', 'lambda_l1': 55.77926104545281, 'lambda_l2': 136.07741184272533, 'max_depth': 3}. Best is trial 0 with value: 1.0813957565304533.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[I 2023-09-06 17:52:02,293] Trial 97 finished with value: 1.0032764452105891 and parameters: {'learning_rate': 0.7593621331936926, 'num_leaves': 43, 'tree_learner': 'serial', 'lambda_l1': 72.38630228517088, 'lambda_l2': 154.50329903014307, 'max_depth': 9}. Best is trial 88 with value: 0.9858088852159445.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000831 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.751225


[I 2023-09-06 17:52:03,797] Trial 98 finished with value: 1.0712366894552237 and parameters: {'learning_rate': 0.8048851265966127, 'num_leaves': 19, 'tree_learner': 'serial', 'lambda_l1': 72.92976381799656, 'lambda_l2': 149.0805597506008, 'max_depth': 10}. Best is trial 88 with value: 0.9858088852159445.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 456
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.743835
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2023-09-06 17:52:05,702] Trial 99 finished with value: 1.155012937277813 and parameters: {'learning_rate': 0.9139429923116611, 'num_leaves': 25, 'tree_learner': 'serial', 'lambda_l1': 84.63513560669212, 'lambda_l2': 153.15401489612321, 'max_depth': 10}. Best is trial 88 with value: 0.9858088852159445.


In [12]:
print("Best trial: {}".format(study.best_trial.number))
print("Best params: {}".format(study.best_trial.params))
print("Best score: {}".format(study.best_trial.value)) 

Best trial: 88
Best params: {'learning_rate': 0.7762661554061182, 'num_leaves': 39, 'tree_learner': 'voting', 'lambda_l1': 85.68378855219873, 'lambda_l2': 154.16051855835047, 'max_depth': 9}
Best score: 0.9858088852159445


In [14]:
df_train, df_val = train_test_split(df, test_size=0.2)
    
col = "n"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

categorical_list = [
                    "Month",
                    "shopID",
                    "Cat",
                    "itemID",
                    ]

model = lgb.train(study.best_trial.params, 
                    trains, 
                    valid_sets=valids, 
                    categorical_feature=categorical_list, 
                    num_boost_round=1000, 
                    )

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.748942
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [6]:
'''
vals = model.predict(val_x)
rmse = np.sqrt(mse(val_y, vals))
rmse
#モデル構築で帰ってきたRMSE値と一致してるか
'''

'\nvals = model.predict(val_x)\nrmse = np.sqrt(mse(val_y, vals))\nrmse\n#モデル構築で帰ってきたRMSE値と一致してるか\n'

In [17]:
#いよいよテストデータを予測してsubmit.csvに書き出し
df_test = pd.read_csv("test_meanprice.csv")
#df_test = tf.data_pre(df_test)

predict = model.predict(df_test)
df_test["n"] = predict
#df_test["y"].to_csv("submit.csv", header = None) #応募用ファイルはヘッダーなし

In [18]:
#lightgbmのfeature_importance関数を使って特徴量重要度を見てみる
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
MeanPrice,1292
itemID,832
shopID,535
Month,432
Cat,5


In [19]:
df_test["n"].to_csv("submit9.csv", index=True, header=False)